# INTRO

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install scikit-plot

from collections import defaultdict

In [2]:
import sys
sys.path.append('./')

from sklearn.naive_bayes import GaussianNB, CategoricalNB

from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preparation

lettura e stampa del training set

In [4]:
dataset="/content/drive/My Drive/Machine Learning Project/monks/monks-2_train.csv"
df_train = pd.DataFrame(pd.read_csv(dataset))
df_train.head()

,classe,a1,a2,a3,a4,a5,a6
0,0,1,1,1,1,2,2
1,0,1,1,1,1,4,1
2,0,1,1,1,2,1,1
3,0,1,1,1,2,1,2
4,0,1,1,1,2,2,1


In [5]:
# label encoding the data
from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
  
df_train['a1']= le.fit_transform(df_train['a1'])
df_train['a2']= le.fit_transform(df_train['a2'])
df_train['a3']= le.fit_transform(df_train['a3'])
df_train['a4']= le.fit_transform(df_train['a4'])
df_train['a5']= le.fit_transform(df_train['a5'])
df_train['a6']= le.fit_transform(df_train['a6'])

y2 = pd.get_dummies(df_train.a1, prefix='a1')
y3 = pd.get_dummies(df_train.a2, prefix='a2')
y4 = pd.get_dummies(df_train.a3, prefix='a3')
y5 = pd.get_dummies(df_train.a4, prefix='a4')
y6 = pd.get_dummies(df_train.a5, prefix='a5')
y7 = pd.get_dummies(df_train.a6, prefix='a6')

df_train = df_train.drop("a1", axis=1)
df_train = df_train.drop("a2", axis=1)
df_train = df_train.drop("a3", axis=1)
df_train = df_train.drop("a4", axis=1)
df_train = df_train.drop("a5", axis=1)
df_train = df_train.drop("a6", axis=1)


frames = [df_train,y2,y3,y4,y5,y6,y7]

df_train = pd.concat(frames, axis=1, join='inner')
display(df_train)

,classe,a1_0,a1_1,a1_2,a2_0,a2_1,a2_2,a3_0,a3_1,a4_0,a4_1,a4_2,a5_0,a5_1,a5_2,a5_3,a6_0,a6_1
0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1
1,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,1,0
2,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0
3,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1
4,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,1,0
165,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1
166,1,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,1,0
167,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,1,0


lettura e stampa del test set

In [6]:
dataset="/content/drive/My Drive/Machine Learning Project/monks/monks-2_test.csv"
df_test = pd.DataFrame(pd.read_csv(dataset))
df_test.head()

,classe,a1,a2,a3,a4,a5,a6
0,0,1,1,1,1,1,1
1,0,1,1,1,1,1,2
2,0,1,1,1,1,2,1
3,0,1,1,1,1,2,2
4,0,1,1,1,1,3,1


In [7]:
# label encoding the data
from sklearn.preprocessing import LabelEncoder
  
le = LabelEncoder()
  
df_test['a1']= le.fit_transform(df_test['a1'])
df_test['a2']= le.fit_transform(df_test['a2'])
df_test['a3']= le.fit_transform(df_test['a3'])
df_test['a4']= le.fit_transform(df_test['a4'])
df_test['a5']= le.fit_transform(df_test['a5'])
df_test['a6']= le.fit_transform(df_test['a6'])

y2 = pd.get_dummies(df_test.a1, prefix='a1')
y3 = pd.get_dummies(df_test.a2, prefix='a2')
y4 = pd.get_dummies(df_test.a3, prefix='a3')
y5 = pd.get_dummies(df_test.a4, prefix='a4')
y6 = pd.get_dummies(df_test.a5, prefix='a5')
y7 = pd.get_dummies(df_test.a6, prefix='a6')

df_test = df_test.drop("a1", axis=1)
df_test = df_test.drop("a2", axis=1)
df_test = df_test.drop("a3", axis=1)
df_test = df_test.drop("a4", axis=1)
df_test = df_test.drop("a5", axis=1)
df_test = df_test.drop("a6", axis=1)


frames = [df_test,y2,y3,y4,y5,y6,y7]

df_test = pd.concat(frames, axis=1, join='inner')
display(df_test)

,classe,a1_0,a1_1,a1_2,a2_0,a2_1,a2_2,a3_0,a3_1,a4_0,a4_1,a4_2,a5_0,a5_1,a5_2,a5_3,a6_0,a6_1
0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,0
1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1
2,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0
3,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,1
4,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,1
428,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0
429,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1
430,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,1,1,0


divisione in training set e test set

In [8]:
class_name = 'classe'

attributes = [col for col in df_train.columns if col != class_name]
X_train = df_train[attributes].values
y_train = df_train[class_name]

attributes = [col for col in df_test.columns if col != class_name]
X_test = df_test[attributes].values
y_test = df_test[class_name]

# CATEGORIAL BAYES UTILIZZO SOLO CAMPI CATEGORIALI RIMUOVO CONTINUI

In [9]:
clf = CategoricalNB()
clf.fit(X_train, y_train)

CategoricalNB(alpha=1.0, class_prior=None, fit_prior=True)

testo il classificatore categoriale sui nuovi dati considerati

In [10]:
y_pred = clf.predict(X_train)
print('Accuracy %s' % accuracy_score(y_train, y_pred))
print('F1-score %s' % f1_score(y_train, y_pred, average=None))
print(classification_report(y_train, y_pred))

Accuracy 0.6272189349112426
F1-score [0.72727273 0.41121495]
              precision    recall  f1-score   support

           0       0.67      0.80      0.73       105
           1       0.51      0.34      0.41        64

    accuracy                           0.63       169
   macro avg       0.59      0.57      0.57       169
weighted avg       0.61      0.63      0.61       169



In [11]:
y_pred = clf.predict(X_test)
print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.6111111111111112
F1-score [0.72815534 0.31707317]
              precision    recall  f1-score   support

           0       0.69      0.78      0.73       290
           1       0.38      0.27      0.32       142

    accuracy                           0.61       432
   macro avg       0.53      0.53      0.52       432
weighted avg       0.58      0.61      0.59       432

